<a href="https://colab.research.google.com/github/HongYue1/LightNovel-Crawler-Colab/blob/main/lightnovel_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <font color="#3399ff" size=10><b> How to use:</b></font>
### 1. Click on connect <font color="yellow" size=5><b> connect </b></font>on the top right and wait untill it finishs connecting.
### 2. Before running a block, make sure to <font color="red" size=5> <b> fill the required data first. </b></font>
### 3. Run each block, wait for the block to finish before running the next block, the block finishes when you see a green check mark<font color ="#1bc757" size=5><b> ✔</b></font> next to the run button.
###4. After uploading finish you will be given the link as an output.
### 5. Note: to download several novels you don't need to redo all this steps, just download a novel then upload it, then go back to the download section and download the next novel then upload it....etc , <font color="red" size=5> <b> you can only upload one novel a time.</br></font>
### I will try to make a list of working sources here when I'm free.
### <font color="#00c3e3"><b> Look at the buttom of this notebook for the list.</b></font>
# <font color="FF1493" size=5> <b> Enjoy <3 </b> </font>
<font size=1> <The archive and upload cells are From OneClickRun, all credit belongs to their repective owners.><font>

## ⏬ Install LightNovel-crawler

In [ ]:
# @markdown <font size =5 color="yellow"> ⬅️ Click Here to install LightNovel-crawler</font>
%%capture
!pip install -U lightnovel-crawler;

## ⏬ Download Novel

In [ ]:
#@markdown <font size =5 color="yellow">⬅️ Click Here to Download</font>

#@markdown <h4> Enter the Link:
#@markdown <h3> You can use this site, it's working fine: https://www.novelpub.com
Link = ""#@param {type:"string"}
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Advanced Options ⚙️</b></font></h3></center>

#@markdown <h4> Enter the range of the chapters separted by a space or enter <font color="#3399ff" size=5><b>All</b></font>  to download all chapters.
#@markdown <h5> Example to download chapters between 1 and 100 : 1 100
Range = "All"#@param {type:"string"}


Format="epub"

def download():

  Misc_command=" -f --suppress --auto-proxy"


  if(Range=="All"):
    !lncrawl --single -o "/content/novels" -s $Link --format $Format --all $Misc_command
  else:
    !lncrawl --single -o "/content/novels" -s $Link --format $Format --range $Range $Misc_command

download()

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> Archive and Upload

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size =5 color="yellow"> ⬅️ Archive Files and Upload</font>


import os

# Specify the directory path
directory_path = '/content/novels/'+Format

# Get the list of files in the directory
file_list = os.listdir(directory_path)

# Get the first file from the list
first_file = file_list[0]

# Split the file name and extension
file_name_without_extension = os.path.splitext(first_file)[0]


MODE = "ZIP"
FILENAME = file_name_without_extension+'.zip'
PATH_TO_FILE = "/content/novels"
# @markdown <h3>Leave the password empty if not needed.
ARCHIVE_PASSWORD = ""#@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no"


compress = 9

# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else ""
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


########################################################
import os, subprocess, json

wd = 'tools/multiupload'
progName = 'go-ul_linux_x64'
os.makedirs(wd, exist_ok=True)
try:
  if not os.path.exists(wd+'/'+progName):
    cmd = !cd $wd;wget https://github.com/bunnykek/go-upload/releases/download/tag/$progName
    cmd = !cd $wd;chmod 755 $progName
  if not cmd:
    print("Requirements installed successfully.")
  else:
    print(cmd, 'Requirements installed unsuccessfully', sep='\n')
except Exception as e:
  print('Error: ', e)

##############################################

#@markdown <h4> You can change the filehost, but transfer.sh is fast and easy.

FileHost = "transfersh" #@param ['catbox', 'fileio', 'filemail', 'gofile', 'krakenfiles', 'letsupload', 'megaup', 'mixdrop', 'pixeldrain', 'racaty', 'transfersh', 'uguu', 'wetransfer', 'workupload']
File_Path = "/content/" + file_name_without_extension + ".zip"
filehost, infile = FileHost, File_Path

message = """
Name : {0}
Host : {1}
URL  : {2}"""
cwd = 'tools/multiupload/'
def upload(filehost:str, filePath: str):
    subprocess.Popen(["./go-ul_linux_x64", filehost,
                     '-f', infile, '-j', 'response.json'],
                     cwd=cwd).wait()

    response = json.load(open(cwd+'response.json'))

    if not response['jobs'][-1]['ok']:
        print(f"{response['jobs'][-1]['error_text']}")
    else:
        print(message.format(response['jobs'][-1]['filename'], filehost, response['jobs'][-1]['url']))

upload(filehost, infile)



# <font color ="#3399ff" size = 6><b> List of working sources:</b></font> <font color="red" size=6><b>(Incomplete)</b></font>

1- <a size = 5 href="https://www.novelpub.com"> Novel Pub</a> Mostly up to date <br />
2-  <a size = 5 href="https://novelnext.com"> Novel Next</a> Very fast